# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2b4cedffd0>
├── 'a' --> tensor([[-1.0113,  0.1614,  1.0607],
│                   [-1.6470,  0.3665,  0.5079]])
└── 'x' --> <FastTreeValue 0x7f2b4cedff70>
    └── 'c' --> tensor([[ 0.6322,  1.0348, -1.6160, -0.7140],
                        [ 2.2341,  0.3842,  1.3844, -0.7391],
                        [-0.5186, -1.4723, -0.3009, -1.8914]])

In [4]:
t.a

tensor([[-1.0113,  0.1614,  1.0607],
        [-1.6470,  0.3665,  0.5079]])

In [5]:
%timeit t.a

78 ns ± 1.96 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2b4cedffd0>
├── 'a' --> tensor([[-1.6984,  1.1307,  0.1724],
│                   [ 0.5878,  0.9564,  0.8859]])
└── 'x' --> <FastTreeValue 0x7f2b4cedff70>
    └── 'c' --> tensor([[ 0.6322,  1.0348, -1.6160, -0.7140],
                        [ 2.2341,  0.3842,  1.3844, -0.7391],
                        [-0.5186, -1.4723, -0.3009, -1.8914]])

In [7]:
%timeit t.a = new_value

85.1 ns ± 1.97 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0113,  0.1614,  1.0607],
               [-1.6470,  0.3665,  0.5079]]),
    x: Batch(
           c: tensor([[ 0.6322,  1.0348, -1.6160, -0.7140],
                      [ 2.2341,  0.3842,  1.3844, -0.7391],
                      [-0.5186, -1.4723, -0.3009, -1.8914]]),
       ),
)

In [10]:
b.a

tensor([[-1.0113,  0.1614,  1.0607],
        [-1.6470,  0.3665,  0.5079]])

In [11]:
%timeit b.a

67.6 ns ± 0.881 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.2617,  0.1541, -0.5914],
               [ 0.8324,  0.1154,  0.8502]]),
    x: Batch(
           c: tensor([[ 0.6322,  1.0348, -1.6160, -0.7140],
                      [ 2.2341,  0.3842,  1.3844, -0.7391],
                      [-0.5186, -1.4723, -0.3009, -1.8914]]),
       ),
)

In [13]:
%timeit b.a = new_value

629 ns ± 5.81 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

989 ns ± 9.95 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.2 µs ± 222 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

294 µs ± 10.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

280 µs ± 15.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2a9816eb50>
├── 'a' --> tensor([[[-1.0113,  0.1614,  1.0607],
│                    [-1.6470,  0.3665,  0.5079]],
│           
│                   [[-1.0113,  0.1614,  1.0607],
│                    [-1.6470,  0.3665,  0.5079]],
│           
│                   [[-1.0113,  0.1614,  1.0607],
│                    [-1.6470,  0.3665,  0.5079]],
│           
│                   [[-1.0113,  0.1614,  1.0607],
│                    [-1.6470,  0.3665,  0.5079]],
│           
│                   [[-1.0113,  0.1614,  1.0607],
│                    [-1.6470,  0.3665,  0.5079]],
│           
│                   [[-1.0113,  0.1614,  1.0607],
│                    [-1.6470,  0.3665,  0.5079]],
│           
│                   [[-1.0113,  0.1614,  1.0607],
│                    [-1.6470,  0.3665,  0.5079]],
│           
│                   [[-1.0113,  0.1614,  1.0607],
│                    [-1.6470,  0.3665,  0.5079]]])
└── 'x' --> <FastTreeValue 0x7f2a9816eb80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

44.4 µs ± 846 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2a981a0d60>
├── 'a' --> tensor([[-1.0113,  0.1614,  1.0607],
│                   [-1.6470,  0.3665,  0.5079],
│                   [-1.0113,  0.1614,  1.0607],
│                   [-1.6470,  0.3665,  0.5079],
│                   [-1.0113,  0.1614,  1.0607],
│                   [-1.6470,  0.3665,  0.5079],
│                   [-1.0113,  0.1614,  1.0607],
│                   [-1.6470,  0.3665,  0.5079],
│                   [-1.0113,  0.1614,  1.0607],
│                   [-1.6470,  0.3665,  0.5079],
│                   [-1.0113,  0.1614,  1.0607],
│                   [-1.6470,  0.3665,  0.5079],
│                   [-1.0113,  0.1614,  1.0607],
│                   [-1.6470,  0.3665,  0.5079],
│                   [-1.0113,  0.1614,  1.0607],
│                   [-1.6470,  0.3665,  0.5079]])
└── 'x' --> <FastTreeValue 0x7f2a981a0970>
    └── 'c' --> tensor([[ 0.6322,  1.0348, -1.6160, -0.7140],
                        [ 2.2341,  0.3842,  1.3844, -0.7391],
                 

In [23]:
%timeit t_cat(trees)

42.4 µs ± 862 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

85.1 µs ± 801 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.6322,  1.0348, -1.6160, -0.7140],
                       [ 2.2341,  0.3842,  1.3844, -0.7391],
                       [-0.5186, -1.4723, -0.3009, -1.8914]],
              
                      [[ 0.6322,  1.0348, -1.6160, -0.7140],
                       [ 2.2341,  0.3842,  1.3844, -0.7391],
                       [-0.5186, -1.4723, -0.3009, -1.8914]],
              
                      [[ 0.6322,  1.0348, -1.6160, -0.7140],
                       [ 2.2341,  0.3842,  1.3844, -0.7391],
                       [-0.5186, -1.4723, -0.3009, -1.8914]],
              
                      [[ 0.6322,  1.0348, -1.6160, -0.7140],
                       [ 2.2341,  0.3842,  1.3844, -0.7391],
                       [-0.5186, -1.4723, -0.3009, -1.8914]],
              
                      [[ 0.6322,  1.0348, -1.6160, -0.7140],
                       [ 2.2341,  0.3842,  1.3844, -0.7391],
                       [-0.5186, -1.4723, -0.3009, -1.8914]],

In [26]:
%timeit Batch.stack(batches)

105 µs ± 1.75 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.6322,  1.0348, -1.6160, -0.7140],
                      [ 2.2341,  0.3842,  1.3844, -0.7391],
                      [-0.5186, -1.4723, -0.3009, -1.8914],
                      [ 0.6322,  1.0348, -1.6160, -0.7140],
                      [ 2.2341,  0.3842,  1.3844, -0.7391],
                      [-0.5186, -1.4723, -0.3009, -1.8914],
                      [ 0.6322,  1.0348, -1.6160, -0.7140],
                      [ 2.2341,  0.3842,  1.3844, -0.7391],
                      [-0.5186, -1.4723, -0.3009, -1.8914],
                      [ 0.6322,  1.0348, -1.6160, -0.7140],
                      [ 2.2341,  0.3842,  1.3844, -0.7391],
                      [-0.5186, -1.4723, -0.3009, -1.8914],
                      [ 0.6322,  1.0348, -1.6160, -0.7140],
                      [ 2.2341,  0.3842,  1.3844, -0.7391],
                      [-0.5186, -1.4723, -0.3009, -1.8914],
                      [ 0.6322,  1.0348, -1.6160, -0.7140],
                   

In [28]:
%timeit Batch.cat(batches)

194 µs ± 3.24 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

598 µs ± 11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
